In [2]:
!pip install langgraph langchain_openai langchain_community

  Using cached langchain_core-0.2.9-py3-none-any.whl.metadata (6.0 kB)
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_community-0.2.5-py3-none-any.whl.metadata (2.5 kB)
  Using cached langchain-0.2.5-py3-none-any.whl.metadata (7.0 kB)
  Using cached langchain_text_splitters-0.2.1-py3-none-any.whl.metadata (2.2 kB)
Using cached langchain_community-0.2.5-py3-none-any.whl (2.2 MB)
Using cached langchain-0.2.5-py3-none-any.whl (974 kB)
Using cached langchain_core-0.2.9-py3-none-any.whl (321 kB)
Using cached langchain_text_splitters-0.2.1-py3-none-any.whl (23 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.52
    Uninstalling langchain-core-0.1.52:
      Successfully uninstalled langchain-core-0.1.52
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.0.2
 

In [4]:
import os
from dotenv import load_dotenv

_ = load_dotenv()


In [7]:
from typing import Annotated, Literal, TypedDict

from langchain_core.messages import HumanMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langgraph.checkpoint import MemorySaver
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

# Define the tools for the agent to use
tools = [TavilySearchResults(max_results=1)]
tool_node = ToolNode(tools)

model = ChatOpenAI(temperature=0).bind_tools(tools)

# Define the function that determines whether to continue or not
def should_continue(state: AgentState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END


# Define the function that calls the model
def call_model(state: AgentState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", 'agent')

# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable.
# Note that we're (optionally) passing the memory when compiling the graph
app = workflow.compile(checkpointer=checkpointer)

# Use the Runnable
final_state = app.invoke(
    {"messages": [HumanMessage(content="what is the weather in sf")]},
    config={"configurable": {"thread_id": 42}}
)

final_state["messages"][-1].content

'You can check the current and future weather conditions for San Francisco, CA on AccuWeather. Here is the [link](https://www.accuweather.com/en/us/san-francisco/94103/weather-forecast/347629) to get detailed information about the temperature, precipitation, wind, air quality, and more.'